In this short tutorial we will install and use [MARTINI](https://martini.readthedocs.io/en/latest/), an analysis package for creating mock HI-data cubes similar to radio interferometer data, written by Kyle Oman (kyle.a.oman@durham.ac.uk). This example uses input from the [FIRE-2](https://ui.adsabs.harvard.edu/abs/2018MNRAS.480..800H/abstract) simulations. The data are publicly available and hosted at [flathub.flatironinstitute.org/fire](https://flathub.flatironinstitute.org/fire).

![MARTINI](https://github.com/kyleaoman/martini/raw/main/martini_banner.png)

MARTINI is a modular package for the creation of synthetic resolved HI line observations (data cubes) of smoothed-particle hydrodynamics simulations of galaxies. The various aspects of the mock-observing process are divided logically into sub-modules handling the data cube, source, beam, noise, spectral model and SPH kernel. MARTINI is object-oriented: each sub-module provides a class (or classes) which can be configured as desired. For most sub-modules, base classes are provided to allow for straightforward customization. Instances of each sub-module class are given as parameters to the Martini class; a mock observation is then constructed by calling a handful of functions to execute the desired steps in the mock-observing process.

This tutorial focuses on particulars related to working with the FIRE simulations. More general information is available in the MARTINI documentation, [hosted on ReadTheDocs](https://martini.readthedocs.io/en/latest/).

## Installation

MARTINI requires `python3` version `3.8` or higher.

The following command will use `pip` to download and install [MARTINI from pypi](https://pypi.org/project/astromartini/):

In [ ]:
import sys

# WAITING ON A. WETZEL TO RELEASE DEPENDENCIES AS PACKAGE
# !{sys.executable} -m pip install "astromartini[firesource]==2.1.3"

If you do not have superuser permissions or use a virtual environment, you may wish to add the --user flag.
With this command required dependencies will be fetched and installed automatically. Watch for error messages during installation. For greater control you may also install the dependencies by hand. These are: numpy, astropy, scipy and h5py.

We'll also install a few other packages used in this notebook:

In [ ]:
!{sys.executable} -m pip install requests matplotlib beautifulsoup4

This cell may be needed in some cases to display figures below:

In [ ]:
%matplotlib inline

Let's check that we can `import martini`:

In [ ]:
import martini

If this produces errors, you may need to restart the Python kernel of this notebook so that it sees the recently installed packages (Kernel -> Restart in the menubar).

We can run MARTINI's built-in demo to check that all of the basic functionality works:

In [ ]:
from martini import demo

demo()

When run successfully, this will make a mock observation of a very simple analytic disc model and write some output to the working directory. Rather than inspect this toy model, let's look at a "real" simulation...

## FIRE Data

This example uses data from the FIRE-2 simulations. These data are hosted at [https://flathub.flatironinstitute.org/fire](https://flathub.flatironinstitute.org/fire).

In this example, we will use the [`m11e_res7100`](https://users.flatironinstitute.org/~chayward/fire2_public_release/core/m11e_res7100) simulation (80 GB). The following cell will download it directly, but note that a [Globus](https://www.globus.org/) endpoint is also provided (see [FlatHUB page](https://flathub.flatironinstitute.org/fire) for details) that can make transfers substantially faster:

In [ ]:
import os
import urllib
import re
import requests
from tqdm import tqdm
import bs4

data_url = "https://users.flatironinstitute.org/~chayward/fire2_public_release/core/"
simulation_directory = "m11e_res7100/"


def direct_download(url, fname):
    with requests.get(url) as r:
        r.raise_for_status()
        with open(fname, "wb") as f:
            f.write(r.content)


def chunked_download(url, fname):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size_in_bytes = int(r.headers.get("content-length", 0))
        chunk_size = 8192
        progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
        with open(fname, "wb") as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                progress_bar.update(len(chunk))
                f.write(chunk)
    return


def directory_download(url_base, directory, chunked_if_larger_than=1024 * 1024 * 10):
    dir_url = urllib.parse.urljoin(url_base, directory)
    print(directory)
    os.mkdir(directory)
    r = requests.get(dir_url)
    data = bs4.BeautifulSoup(r.text, "html.parser")
    for link in data.find_all("a"):
        link_url = urllib.parse.urljoin(dir_url, link["href"])
        if ".." in link["href"]:  # parent directory, skip
            continue
        elif "." in link["href"]:  # a file, download
            target_file = urllib.parse.urljoin(directory, link["href"])
            with requests.head(link_url) as h:
                h.raise_for_status()
                total_size_in_bytes = int(h.headers.get("content-length", 0))
                print(target_file)
                if total_size_in_bytes > chunked_if_larger_than:
                    chunked_download(link_url, target_file)
                else:
                    direct_download(link_url, target_file)
        else:  # a directory, recurse
            directory_download(url_base, urllib.parse.urljoin(directory, link["href"]))


if os.path.isdir(simulation_directory):
    print("Found directory matching simulation_directory locally, skipping download.")
else:
    directory_download(data_url, simulation_directory)

If you have downloaded the data manually, edit this cell to specify the location of the base directory:

In [ ]:
simulation_directory = (
    simulation_directory  # e.g. simulation_directory = "/path/to/m11e_res7100/"
)

The i/o tools for FIRE simulations allow a few ways of defining a snapshot, by its `"index"`, `"redshift"` or `"scalefactor"`. Here we'll use the `"redshift"` method and specify redshift `0`; the i/o tools will automatically select the closest snapshot to z=0. Since the FIRE suite is a set of zoom simulations (at least those in the FIRE-2 [public data release](https://flathub.flatironinstitute.org/fire)), the galaxy of interest is usually the first in the catalogue, corresponding to `host_number = 1`. Subsequent galaxies in the catalogue can be selected by increasing this number.

In [ ]:
snapshot = ("redshift", 0)
host_number = 1

## FIRE Example

First, import some modules from Martini, and the units module from astropy.

In [ ]:
from martini.sources import FIRESource
from martini import DataCube, Martini
from martini.beams import GaussianBeam
from martini.noise import GaussianNoise
from martini.spectral_models import GaussianSpectrum
from martini.sph_kernels import CubicSplineKernel
import astropy.units as U

The different martini sub-modules need to be initialized, 
see the [full documentation](https://martini.readthedocs.io/en/latest/) for details of all configuration options. A few suggested best-practices specific to FIRE are outlined below.

### SOURCE

The `FIRESource` module is built around the i/o tools for the FIRE simulations, which are built assuming that the target object in a zoom simulation is an object of interest. The default (and only) behaviour is to read in all of the particles of a given type in the snapshot. Initializing the source module will therefore have a memory usage similar to the size of the gas particle arrays in the snapshot (in this case about ?? GB). The memory footprint can be reduced by reducing precision to 32-bit floats with `convert_float32=True`.

Galaxy positions and velocities are assigned by default using a "shrinking spheres" algorithm weighted by the particle masses (`assign_hosts="mass"`), but other schemes are also available (potential-weighted with `assign_hosts="potential"`, metallicity-weighted with `assign_hosts="massfraction.metals"`, based on smoothed position across snapshots with `assign_hosts="track"`, read directly from the Rockstar halo finder catalogues with `assign_hosts="halo"`; setting `assign_hosts=True` will attempt to find a sensible default by first trying `"track"`, then `"mass"`).

The `FIRESource` assumes the molecular gas partitioning scheme described in [Orr et al. (2018)](https://ui.adsabs.harvard.edu/abs/2018MNRAS.478.3653O/abstract), Appendix B.

The i/o tools print some messages as data is read, at present there does not seem to be an option to silence these.

In [ ]:
source = FIRESource(
    simulation_directory=simulation_directory,
    snapshot=snapshot,
    host_number=host_number,
    assign_hosts="mass",
    convert_float32=False,
    distance=5.0 * U.Mpc,
    vpeculiar=0 * U.km / U.s,
    rotation={"rotmat": np.eye(3)},
    ra=0.0 * U.deg,
    dec=0.0 * U.deg,
)

The rotation argument above has been set to the identity matrix, so the source has the (random) orientation that it has within the simulation volume. The source class includes a function to make a quick plot to get an idea of the source's appearance:

In [ ]:
preview_fig_unrotated = source.preview(title="unrotated")

The preview function defaults to apertures in position and velocity that enclose all particles in the source (and therefore the entire zoom simulation), so this preview emphasizes the diffuse circumgalactic and intergalactic gas. The apertures can be set manually using the `lim` and `vlim` keywords to set the maximum absolute offsets in position and velocity relative to the source centre to be plotted. For example, restricting the aperture to 50kpc and 300km/s makes the disc more clearly visible.

In [ ]:
preview_fig_unrotated_zoom = source.preview(
    title="unrotated, zoomed-in",
    lim=50 * U.kpc,
    vlim=300 * U.km / U.s,
)

This randomly-oriented viewing angle seems to be moderately inclined with respect to the disc. The source can be rotated to a different orientation. MARTINI's tool for quick/approximate manipulation of the orientation of the source aligns the source based on its angular momentum vector ("L"), for example:

In [ ]:
source.rotate(L_coords=(60 * U.deg, 90 * U.deg))

The rotation configuration takes an inclination (here 60deg) and rotation about the pole (here 90deg, relative to an arbitrary reference direction). The code attempts to
automatically align the galactic disk in the y-z plane by aligning
the angular momentum along the x-axis. The polar rotation is then
applied, and finally the disc inclined by a rotation around the
y-axis (the line of sight is along the x-axis). The automatic
alignment will work for typical reasonably isolated discs (no other galaxies with HI in the zoom region), but will
struggle when companions are present, when the angular momentum axis
is a poor tracer of the disc plane, and especially for satellites or galaxies in pairs.

In [ ]:
preview_fig_rotated_zoomed_in = source.preview(
    title="rotated, zoomed-in",
    lim=50 * U.kpc,
    vlim=300 * U.km / U.s,
)

If finer control of the orientation is needed, derive the transformation from the simulation box coordinates (see [the documentation](https://martini.readthedocs.io/en/latest/) for examples) to the desired coordinates for the 'observation', keeping in mind that the line of sight is along the x-axis. This rotation matrix can then be passed to the rotate function as `rotmat=np.eye(3)` (here the identity rotation matrix used as a place holder). The rotation can also be provided when the source is initialized by using the `rotation` keyword argument.

A common problem is deriving the inverse transform instead of the forward transform, if unexpected results are obtained, first try passing the transpose of the rotation matrix.

### DATACUBE

It is usually advisable to set the centre of the cube to track the
centre of the source, as illustrated below. Note that the source
systemic velocity is set according to the distance, peculiar velocity, and Hubble's law.
These values can instead be set explicitly, if desired. A datacube
with 128x128 pixels usually takes a few minutes, depending on the number of particles. 1024x1024 can take
several hours. The number of channels has less influence on the
runtime. Most of the runtime is spent when `M.insert_source_in_cube` is
called below.

In [ ]:
datacube = DataCube(
    n_px_x=384,
    n_px_y=384,
    n_channels=50,
    px_size=10.0 * U.arcsec,
    channel_width=5.0 * U.km * U.s**-1,
    velocity_centre=source.vsys,
    ra=source.ra,
    dec=source.dec,
)

### BEAM

It is usually advisable to set the beam size to be ~3x the pixel
size. Note that the data cube is padded according to the size of the
beam, this usually results in the number of pixel rows printed in the
progress messages to differ from the requested dimensions. The
padding is required for accurate convolution with the beam, but
contains incorrect values after convolution and is discarded to
produce the final data cube of the requested size.

In [ ]:
beam = GaussianBeam(
    bmaj=30.0 * U.arcsec,
    bmin=30.0 * U.arcsec,
    bpa=0.0 * U.deg,
    truncate=3.0,
)

### NOISE

The noise is normally added before convolution with the beam (as
below in this example). The rms value passed is that corresponding to the desired noise level in the final data cube, in Jy/beam or equivalent units. To obtain consistent random realisations each time the code is run, we provide a random seed (integer).

In [ ]:
noise = GaussianNoise(
    rms=3.0e-8 * U.Jy * U.beam**-1,
    seed=0,
)

### SPECTRAL MODEL

The `thermal` mode estimates the HI line width for each particle based on its properties (temperature, etc.). The 'subgrid' velocity dispersion can also be fixed to a constant value, e.g. `sigma=7 * U.km / U.s`.

In [ ]:
spectral_model = GaussianSpectrum(sigma="thermal")

The calculation of the spectra (that will happen when the `Martini` module is initialized below) can be done in parallel by providing a keyword argument `ncpu=N`, where `N` is the number of CPUs to use. However, the details of the implementation mean that for small numbers of particles running in parallel tends to slow down the calculation, so turning this on should be done with care. Significant speedups can be expected when the particle count is very large.

### SPH KERNEL

The FIRE simulations are meshless finite mass (MFM), not smoothed particle hydrodynamics (SPH) simulations. MARTINI is strictly speaking designed for SPH simulations, but can still provide a good approximation of non-SPH simulations by representing mesh cells as SPH particles assuming their centroids and characteristic smoothing lengths. The MFM scheme in FIRE uses a cubic spline kernel function (although this is not an SPH kernel), so we may as well use MARTINI's `CubicSplineKernel` module for the approximation.

In [ ]:
sph_kernel = CubicSplineKernel()

### MARTINI

Now set up the configuration:

In [ ]:
M = Martini(
    source=source,
    datacube=datacube,
    beam=beam,
    noise=noise,
    spectral_model=spectral_model,
    sph_kernel=sph_kernel,
)

Similar to previewing the source, we can make a preview here. Now the extent of the datacube is overlaid with a red box.

In [ ]:
M.preview(
    point_scaling="fixed", vlim=180 * U.km / U.s
)  # can experiment with parameters here

The main source insertion loop, that is the most computationally demanding step, can be run in parallel if the `multiprocess` package is installed (not to be confused with `multiprocessing`, which is normally included with python!). Edit the cell below to use more than 1 CPU core if this package is installed.

In [ ]:
ncpu = 1  # can be >1 if multiprocess package is installed

If we're happy with the preview, we're ready to call the functions to make the actual mock observation.

In [ ]:
M.insert_source_in_cube(ncpu=ncpu)
M.add_noise()
M.convolve_beam()

You may notice that the number of pixels in the progress counter differs from the number defined in the DataCube module. This is because convolution with the beam requires some padding, which is ultimately filled with nonsense and discarded.

To write the results: two output formats are available, depending on preference. Both
formats are self-documenting, via FITS header keywords and HDF5
attributes, respectively. For HDF5 output, the beam image is included
in the same file. (If you do not have h5py installed, comment out the call to `write_hdf5`.)

In [ ]:
M.write_fits("fire_martini_demo.fits")
M.write_beam_fits("fire_martini_demo_beam.fits")
M.write_hdf5("fire_martini_demo.hdf5")

### Inspect the results (FITS)

Let's load the FITS file that MARTINI produced and take a quick look.

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS

with fits.open("fire_martini_demo.fits") as f:
    cube_wcs = WCS(f[0].header)
    flux_cube = f[0].data * U.Unit(f[0].header["BUNIT"])
    n_channels = cube_wcs.pixel_shape[cube_wcs.wcs.spec]
    vch = np.array(cube_wcs.sub(axes=[3]).all_pix2world(np.arange(n_channels), 0))[
        0
    ] * U.Unit(cube_wcs.world_axis_units[cube_wcs.wcs.spec])
    vch = vch - source.vsys

In [ ]:
flux_cube.shape

Let's examine one of the velocity channels:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.clf()
ax = fig.add_subplot(1, 1, 1, aspect="equal", projection=cube_wcs.celestial)

# choose units for plotting, not necessarily the units data are stored in:
flux_unit = U.Jy / U.beam

plt.imshow(flux_cube[27, ...].to_value(flux_unit), cmap="Greys")
plt.xlabel("RA")
plt.ylabel("Dec")
plt.colorbar(label=f"Flux [{flux_unit}]");

And do a quick plot of the first three moments:

In [ ]:
import numpy as np

# choose plotting units
mom0_unit = U.Jy / U.beam
mom1_unit = U.km / U.s
mom2_unit = U.km / U.s

rms = np.std(
    flux_cube[:, :20, :20]
)  # noise in a corner patch where there is little signal
clip = np.where(flux_cube > 5 * rms, 1, 0)
np.seterr(all="ignore")
fig = plt.figure(figsize=(16, 5))
plt.clf()
sp1 = fig.add_subplot(1, 3, 1, aspect="equal", projection=cube_wcs.celestial)
sp2 = fig.add_subplot(1, 3, 2, aspect="equal", projection=cube_wcs.celestial)
sp3 = fig.add_subplot(1, 3, 3, aspect="equal", projection=cube_wcs.celestial)
mom0 = np.sum(flux_cube, axis=0)
mask = np.where(mom0 > 0.002 * U.Jy / U.beam, 1, np.nan)
mom1 = np.sum(flux_cube * clip * vch[:, np.newaxis, np.newaxis], axis=0) / mom0
mom2 = np.sqrt(
    np.sum(
        flux_cube
        * clip
        * np.power(vch[:, np.newaxis, np.newaxis] - mom1[np.newaxis], 2),
        axis=0,
    )
    / mom0
)
im1 = sp1.imshow(mom0.to_value(mom0_unit), cmap="Greys")
plt.colorbar(im1, ax=sp1, label=f"mom0 [{mom0_unit}]")
im2 = sp2.imshow(
    (mom1 * mask).to_value(mom1_unit),
    cmap="RdBu_r",
    vmin=-np.nanmax(np.abs(mom1 * mask)).to_value(mom1_unit),
    vmax=np.nanmax(np.abs(mom1 * mask)).to_value(mom1_unit),
)
plt.colorbar(im2, ax=sp2, label=f"mom1 [{mom1_unit}]")
im3 = sp3.imshow(
    (mom2 * mask).to_value(mom2_unit),
    cmap="magma",
)
plt.colorbar(im3, ax=sp3, label=f"mom2 [{mom2_unit}]")
for sp in sp1, sp2, sp3:
    sp.set_xlabel("RA")
    sp.set_ylabel("Dec")

plt.subplots_adjust(wspace=0.3)

This galaxy has a strongly disturbed morphology, but does display clear signs of rotation. The alignment of the disc, such as it is, looks as expected - the inclination is plausibly about 60 degrees, and the position angle of the central part is horizontal in the figure - in this case the automated orientation function has performed well, though it should never be assumed that this will always be the case!

For complete documentation, more usage examples, and further information, please take a look at the [MARTINI webpage](https://martini.readthedocs.io/en/latest/).

### Inspect the results (HDF5)

Let's load the HDF5 that MARTINI produced and take a quick look.

In [ ]:
import h5py

f = h5py.File("fire_martini_demo.hdf5", "r")

In [ ]:
list(f.keys())

In addition to the mock cube itself (`f["FluxCube"]`), the HDF5 output contains arrays of the same size with the coordinates of the centre of each cell in the cube in RA, Dec and spectral space (`f["RA"]`,`f["Dec"]` and `f["channel_mids"]`). There are also arrays longer by one in each dimension that contain the coordinates of the corners of each cell in the cube (`f["RA_vertices"]`,`f["Dec_vertices"]` and `f["channel_vertices"]`). The latter are convenient for use with the `pcolormesh` plotting function from matplotlib, so we'll read these.

In [ ]:
flux_cube = f["FluxCube"][()] * U.Unit(f["FluxCube"].attrs["FluxCubeUnit"])
ra_vertices = f["RA_vertices"][()] * U.Unit(f["RA_vertices"].attrs["Unit"])
dec_vertices = f["Dec_vertices"][()] * U.Unit(f["RA_vertices"].attrs["Unit"])
spec_vertices = f["channel_vertices"][()] * U.Unit(f["channel_vertices"].attrs["Unit"])
vch = (
    f["velocity_channel_mids"][()] * U.Unit(f["velocity_channel_mids"].attrs["Unit"])
    - source.vsys
)
f.close()

The RA range of our cube straddles the RA=0 boundary, let's shift the `ra_vertices` to a (-180, 180] range for plotting:

In [ ]:
ra_vertices = np.where(
    ra_vertices > 180 * U.deg, ra_vertices - 360 * U.deg, ra_vertices
)

In [ ]:
flux_cube.shape

Let's examine one of the velocity channels:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

channel_slice = np.s_[:, :, 15]  # a slice of the data cube containing one channel

# choose units for plotting, not necessarily the units data are stored in:
ra_unit = U.deg
dec_unit = U.deg
flux_unit = U.Jy / U.beam

plt.pcolormesh(
    ra_vertices[channel_slice].to_value(ra_unit),
    dec_vertices[channel_slice].to_value(dec_unit),
    flux_cube[channel_slice].to_value(flux_unit),
    cmap="Greys",
)
ax.set_xlabel(f"RA [{ra_unit}]")
ax.set_ylabel(f"Dec [{dec_unit}]")
ax.set_xlim(ax.get_xlim()[::-1])
plt.colorbar(label=f"Flux [{flux_unit}]");

And do a quick plot of the first three moments:

In [ ]:
import numpy as np

# choose units for plotting, not necessarily the units data are stored in:
ra_unit = U.deg
dec_unit = U.deg
mom0_unit = U.Jy / U.beam
mom1_unit = U.km / U.s
mom2_unit = U.km / U.s

np.seterr(all="ignore")
fig = plt.figure(figsize=(16, 5))
sp1 = fig.add_subplot(1, 3, 1, aspect="equal")
sp2 = fig.add_subplot(1, 3, 2, aspect="equal")
sp3 = fig.add_subplot(1, 3, 3, aspect="equal")
rms = np.std(
    flux_cube[:16, :16]
)  # noise in a corner patch where there is little signal
clip = np.where(flux_cube > 5 * rms, 1, 0)
mom0 = np.sum(flux_cube, axis=-1)
mask = np.where(mom0 > 0.002 * U.Jy / U.beam, 1, np.nan)
mom1 = np.sum(flux_cube * clip * vch, axis=-1) / mom0
mom2 = np.sqrt(
    np.sum(flux_cube * clip * np.power(vch - mom1[..., np.newaxis], 2), axis=-1) / mom0
)
im1 = sp1.pcolormesh(
    ra_vertices[..., 0].to_value(
        ra_unit
    ),  # pick one channel, coordinates are the same in all of them
    dec_vertices[..., 0].to_value(
        dec_unit
    ),  # pick one channel, coordinates are the same in all of them
    mom0.to_value(mom0_unit),
    cmap="Greys",
)
plt.colorbar(im1, ax=sp1, label=f"mom0 [{mom0_unit}]")
im2 = sp2.pcolormesh(
    ra_vertices[..., 0].to_value(
        ra_unit
    ),  # pick one channel, coordinates are the same in all of them
    dec_vertices[..., 0].to_value(
        dec_unit
    ),  # pick one channel, coordinates are the same in all of them
    (mom1 * mask).to_value(mom1_unit),
    cmap="RdBu_r",
    vmin=-np.nanmax(np.abs(mom1 * mask)).to_value(mom1_unit),
    vmax=np.nanmax(np.abs(mom1 * mask)).to_value(mom1_unit),
)
plt.colorbar(im2, ax=sp2, label=f"mom1 [{mom1_unit}]")
im3 = sp3.pcolormesh(
    ra_vertices[..., 0].to_value(
        ra_unit
    ),  # pick one channel, coordinates are the same in all of them
    dec_vertices[..., 0].to_value(
        dec_unit
    ),  # pick one channel, coordinates are the same in all of them
    (mom2 * mask).to_value(mom2_unit),
    cmap="magma",
    vmin=0,
)
plt.colorbar(im3, ax=sp3, label=f"mom2 [{mom2_unit}]")
for sp in sp1, sp2, sp3:
    sp.set_xlabel(f"RA [{ra_unit}]")
    sp.set_ylabel(f"Dec [{dec_unit}]")
    sp.set_xlim(sp.get_xlim()[::-1])
plt.subplots_adjust(wspace=0.3)

This galaxy has a strongly disturbed morphology, but does display clear signs of rotation. The alignment of the disc, such as it is, looks as expected - the inclination is plausibly about 60 degrees, and the position angle of the central part is horizontal in the figure - in this case the automated orientation function has performed well, though it should never be assumed that this will always be the case!

For complete documentation, more usage examples, and further information, please take a look at the [MARTINI webpage](https://martini.readthedocs.io/en/latest/).